# LlamaIndex

## Overview

This is a Quick Start guide that shows how to use Guardrails alongside LlamaIndex.  As you'll see, the LlamaIndex portion comes directly from their starter examples [here](https://docs.llamaindex.ai/en/stable/getting_started/starter_example/).  Our approach to intergration for LlamaIndex, similar to our LangChain integration, is the make the interaction feel as native to the tool as possible.

## Installation
Install LlamaIndex and a version of Guardrails with LlamaIndex support.

In [ ]:
! pip install llama-index -q
! pip install "guardrails-ai>=0.6.1" -q

Install a couple validators from the Guardrails Hub that we'll use to guard the query outputs.

In [3]:
! guardrails hub install hub://guardrails/detect_pii hub://guardrails/competitor_check --no-install-local-models -q

Installing hub://guardrails/detect_pii...
✅Successfully installed guardrails/detect_pii version 0.0.5!


Installing hub://guardrails/competitor_check...
✅Successfully installed guardrails/competitor_check!




Download some sample data from the LlamaIndex docs.

In [4]:
! mkdir -p ./data
! curl https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt > ./data/paul_graham_essay.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75042  100 75042    0     0   353k      0 --:--:-- --:--:-- --:--:--  354k


## Index Setup

First we'll load some data and build an index as shown in the [starter tutorial](https://docs.llamaindex.ai/en/stable/getting_started/starter_example/)

In [1]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

## Guard Setup

Next we'll create our Guard and assign some validators to check the output from our queries.

In [ ]:
from guardrails import Guard
from guardrails.hub import CompetitorCheck, DetectPII

guard = (
    Guard()
    .use(CompetitorCheck(competitors=["Fortran", "Ada", "Pascal"], on_fail="fix"))
    .use(DetectPII(pii_entities=["PERSON", "EMAIL_ADDRESS"], on_fail="fix"))
)

## Querying The Index

To demonstrate it's plug-and-play capabilities, first we'll query the index un-guarded.

In [3]:
# Use index on it's own
query_engine = index.as_query_engine()
response = query_engine.query("Who is the author and what did they do growing up?")
print(response)

The author is Paul Graham. Growing up, he worked on writing short stories and programming, starting with the IBM 1401 in 9th grade using an early version of Fortran. Later, he transitioned to microcomputers like the TRS-80 and began programming more extensively, creating simple games and a word processor.


Now we'll set up a guarded engine, and re-query the index to see how Guardrails applies the fixes we specified when assigning our validators to the Guard.

In [ ]:
# Use index with Guardrails
from guardrails.integrations.llama_index import GuardrailsQueryEngine

guardrails_query_engine = GuardrailsQueryEngine(engine=query_engine, guard=guard)

response = guardrails_query_engine.query(
    "Who is the author and what did they do growing up?"
)
print(response)

The author is <PERSON>. Growing up, he worked on writing short stories and programming, starting with the IBM 1401 in 9th grade using an early version of [COMPETITOR]. Later, he transitioned to microcomputers like the TRS-80 and Apple II, where he wrote simple games, programs, and a word processor. 


The GuardrailsEngine can also be used with LlamaIndex's chat engine, not just the query engine.

In [ ]:
# For chat engine
from guardrails.integrations.llama_index import GuardrailsChatEngine

chat_engine = index.as_chat_engine()
guardrails_chat_engine = GuardrailsChatEngine(engine=chat_engine, guard=guard)

response = guardrails_chat_engine.chat(
    "Tell me who the author is and what they did growing up."
)
print(response)

The author is <PERSON>. Growing up, he worked on writing short stories and programming. He started with early attempts on an IBM 1401 using [COMPETITOR] in 9th grade. Later, he transitioned to microcomputers, building a Heathkit kit and eventually getting a TRS-80 to write simple games and programs. Despite enjoying programming, he initially planned to study philosophy in college but eventually switched to AI due to a lack of interest in philosophy courses. 
